In [49]:
import pandas as pd 
import numpy as np

In [50]:
#raws
raw_teleoperadoras=pd.read_excel(r"C:\Users\Abraham\Desktop\Solo\Bootcamp\portfolio\PowerBI\ventas_telemarketing.xlsx", skiprows=1)
raw_comerciales=pd.read_excel(r"C:\Users\Abraham\Desktop\Solo\Bootcamp\portfolio\PowerBI\ventas_comerciales.xlsx",skiprows=1)
raw_tecnicos=pd.read_excel(r"C:\Users\Abraham\Desktop\Solo\Bootcamp\portfolio\PowerBI\ventas_tecnicos.xlsx",skiprows=1)

## Tratado teleoperadoras

In [51]:
teleop=raw_teleoperadoras.copy()
teleop["Departamento"]="Teleoperadora"

In [52]:
#extraigo fuente de donde provienen las ventas:
import unicodedata

def normaliza(texto):
    texto = str(texto).lower()
    texto = unicodedata.normalize('NFKD', texto)
    texto = ''.join(c for c in texto if not unicodedata.combining(c))
    return texto

def fuente(text):
    text = normaliza(text)
    if "web" in text:
        return "Web"
    elif "hoy" in text:
        return "Reformashoy"
    elif "chat" in text:
        return "Web"
    elif "renova" in text:
        return "renova"
    elif "mail" in text:
        return "Web"
    elif "pisos" in text:
        return "Rdp"
    else:
        return "Telemarketing"

def fuente (text):
    text=str(text).lower()
    if "web" in text:
        return "Web"
    elif "hoy" in text:
        return "Reformashoy"
    elif "chat" in text:
        return "Web"
    elif "renova" in text:
        return "Renova"
    elif "mail" in text:
        return "Web"
    elif "pisos" in text:
        return "Rdp"
    else:
        return "Telemarketing"

In [53]:
teleop["Fuente"]=teleop["Comercial/Telemarketing/Técnico"].apply(fuente)

In [54]:
#Agrupo entre externos, internos por web y sin web

def origen(text):
    text=str(text).lower()
    if "renova" in text:
        return "externos"
    elif "reformashoy" in text:
        return "externos"
    elif any(word in text for word in ["web", "chat", "mail", "rdp"]):
        return "webs"
    else: 
        return "telemarketing"    
    
         

In [55]:
teleop["Origen"]=teleop["Fuente"].apply(origen)

In [56]:
def nombres (texto):
    texto=str(texto).lower()
    if "rosa" in texto:
     return "Rosa"
    elif "amparo" in texto:
     return "Amparo"
    elif "carmen" in texto:
      return "Maricarmen"
    elif "teresa" in texto:
      return "Teresa"
    elif "narcisa" in texto:
      return "Narcisa"
    elif "dolores" in texto:
      return "Loli"
    elif "loli" in texto:
      return "Loli"
    elif "monica" in texto:
      return "Monica"
    elif "vicky" in texto:
      return "Vicky"
    elif "natalia" in texto:
      return "Natalia"
    else: 
      return texto.capitalize()

In [57]:
teleop["Empleado"]=teleop["Comercial/Telemarketing/Técnico"].apply(nombres)

In [58]:
#2994 totales, 59 daniel
#alternativa: teleop_2=teleop[~(teleop["Empleado"]=='Daniel olivar')]
teleop_2=teleop[(teleop["Empleado"]!='Daniel olivar')]

In [59]:
#Entre las teleoperadoras aparece Daniel, son regularizaciones de obras (descuentos), habría que añadir si son de web etc y los que no dejar como telemarketing
teleop_daniel=teleop[teleop["Empleado"]=='Daniel olivar']


In [60]:
teleop_daniel.drop(["Fuente", "Origen"],axis=1,inplace=True)

c:\Users\Abraham\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [61]:
#Estos datos de teleoperadoras son clave para determinar el origen y la fuente en comerciales y técnicos
puente=teleop_2[["PTO","Fuente", "Origen"]]

In [62]:
puente.drop_duplicates(subset="PTO",inplace=True)


c:\Users\Abraham\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [63]:
teleop_daniel=teleop_daniel.merge(puente,on="PTO",how="left")

In [64]:
teleop_daniel["Fuente"].fillna("Telemarketing",inplace=True)
teleop_daniel["Origen"].fillna("Telemarketing",inplace=True)

In [65]:
teleop_2.shape

(2935, 15)

In [66]:
teleop_total=pd.concat([teleop_2,teleop_daniel], ignore_index=True )

teleop_total.shape

(2994, 15)

In [67]:
puente2=teleop_total[["PTO","Fuente", "Origen"]]
puente2.drop_duplicates(subset="PTO",inplace=True)

## Comerciales

In [68]:
comercial=raw_comerciales.copy()

In [69]:
comercial=comercial.merge(puente2,on="PTO",how="left")

In [70]:
comercial["Departamento"]="Comercial"
comercial["Empleado"]=comercial["Comercial/Telemarketing/Técnico"]

## Tecnicos

In [71]:
tecnico= raw_tecnicos.copy()

In [72]:
tecnico["Departamento"]="Tecnico"
tecnico["Empleado"]=tecnico["Comercial/Telemarketing/Técnico"]

In [73]:
tecnico=tecnico.merge(puente2,on="PTO",how="left")

## Fusión de ficheros y clean

In [74]:
#comprobamos tienen el mismo número de columnas :Ok, 15
tecnico.shape

(2932, 15)

In [75]:
comercial.shape

(2994, 15)

In [76]:
teleop_total.shape

(2994, 15)

In [77]:
ventas_totales=pd.concat([teleop_total,comercial,tecnico],ignore_index=True)

In [78]:
#ventas_totales.drop(['Comercial/Telemarketing/Técnico','Observaciones Pagos' ],axis=1,inplace=True)

In [79]:
ventas_totales["Tipo"].unique()

array(['CALEFACCION VAILLANT', 'REFORMA VIVIENDA 5%',
       'REFORMA DE PINTURA', 'REFORMA DE COCINA',
       'REFORMA BAÑO + MAMPARA', 'CARPINTERIA DE MADERA',
       'REFORMA DE VIVIENDA 3%', 'REFORMA PINTURA Y VARIOS',
       'REFORMA DE VIVIENDA', 'REFORMA DE SUELO Y PINTURA',
       'TRABAJOS VARIOS', 'REFORMA PVC SLAVA', 'DESCUENTO DE OBRA',
       'LICENCIA', 'AMPLIACION DE OBRA', 'CONTENEDOR', 'REFORMA BAÑO',
       'AMPLIACION MUEBLES DE COCINA', 'MUEBLES DE COCINA',
       'LICENCIA Y CONTENEDOR', 'REFORMA COMUNIDAD',
       'REFORMA DE ELECTRICIDAD', 'REFORMA PLATO X PLATO',
       'REFORMA BAÑERA X PLATO', 'REFORMAS PVC LORENTE', nan,
       'AMPLIACION ALUMINIO / PVC', 'BAJANTE COMUNIDAD', 'CALDERA',
       'REFORMA DE PINTURA, ALBAÑILERIA Y PARQUET', 'REFORMA TERRAZA',
       'REFORMA BAÑO Y COCINA', 'AIRE ACONDICIONADO VAILLANT',
       'MUEBLES DE COCINA 3% DCTO', 'REFORMA ALUMINIO NOGALES', 'MAMPARA',
       'REFORMA DE COCINA 3%', 'REFORMA ALUMINIO SLAVA', 'REFORMA T

In [80]:
def tipo (texto):
    texto=str(texto).lower()
    if "reforma" in texto:
        return "Reforma"
    elif "ampliacion" in texto:
        return "Ampliaciones"
    elif "descuento" in texto:
        return "Descuentos"
    elif "calefaccion" in texto:
        return "Calderas"
    elif "aire" in texto:
        return "Aires Acondicionados"
    elif "calentador" in texto:
        return "Calentadores"
    elif "licencia" in texto:
        return "Licencias"
    elif "contenedor" in texto:
        return "Contenedores"
    elif "interiorismo" in texto:
        return "Interiorismo"
    elif "trabajos" in texto:
        return "Reforma"
    else:
        return "Otros trabajos"

In [81]:
ventas_totales["Servicio"]=ventas_totales["Tipo"].apply(tipo)

In [82]:
ventas_totales.columns

Index(['Comercial/Telemarketing/Técnico', 'Fecha Acepta', 'F.P.R.',
       'Fecha Inicio', 'Fecha Entrega', 'PTO', 'Presupuesto Insitu',
       'Dirección', 'Importe', 'Observaciones Pagos', 'Tipo', 'Departamento',
       'Fuente', 'Origen', 'Empleado', 'Servicio'],
      dtype='object')

In [83]:
ventas_totales25=ventas_totales[ventas_totales["Fecha Acepta"].str.contains("2025")]

In [84]:
ventas_totales25["Column1"]=1

c:\Users\Abraham\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [85]:
#excluimos datos de los 300000

filtro = ventas_totales25[~ventas_totales25["PTO"].astype(str).str.startswith("300")]


In [86]:
ventas_totales25["FAcepta fto fecha"]=pd.to_datetime(
    ventas_totales25["Fecha Acepta"].astype(str).str.strip(),
    format="%d/%m/%Y",  # <-- Día / Mes / Año
    errors="coerce"
)


c:\Users\Abraham\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [87]:
mask=ventas_totales25["Servicio"]=="Descuentos"

In [88]:
ventas_totales25_check=ventas_totales25[mask]
ventas_totales25_check

,Comercial/Telemarketing/Técnico,Fecha Acepta,F.P.R.,Fecha Inicio,Fecha Entrega,PTO,Presupuesto Insitu,Dirección,Importe,Observaciones Pagos,Tipo,Departamento,Fuente,Origen,Empleado,Servicio,Column1,FAcepta fto fecha
26,AMPARO MONTOYA,07/04/2025,07/04/2025,14/05/2025,26/02/2025,67128,No,"C/ Dr Fourquet 32, 2º izda - 28012 MADRID (MAD...",-486.00,NaN,DESCUENTO DE OBRA,Teleoperadora,Telemarketing,telemarketing,Amparo,Descuentos,1,2025-04-07
28,AMPARO MONTOYA,10/03/2025,10/04/2025,01/04/2025,25/02/2025,67207,Sí,"C/ Alameda del Valle 23, 3º 19 - 28031 MADRID ...",-1266.00,18/03 Nico lo entg** DCTO SIN FIRMAR X CLTE**S...,DESCUENTO DE OBRA,Teleoperadora,Telemarketing,telemarketing,Amparo,Descuentos,1,2025-03-10
35,AMPARO MONTOYA,30/05/2025,30/05/2025,31/03/2025,16/01/2025,66948,No,"C/ Camino de los Alamillos 14, 1º A - 28891 V...",-336.00,02/06/25 X.O DE GLORIA METO DCTO POR PAGO DE L...,DESCUENTO DE OBRA,Teleoperadora,Telemarketing,telemarketing,Amparo,Descuentos,1,2025-05-30
36,AMPARO MONTOYA,30/05/2025,30/05/2025,10/04/2025,13/02/2025,66999,No,"C/ Europa Avda 17, Portal 8, 2º 2 - 28224 POZU...",-17.75,X.O DE SANDRA METO DCTO** LOS 4 LEDS DEL BAÑO ...,DESCUENTO DE OBRA,Teleoperadora,Telemarketing,telemarketing,Amparo,Descuentos,1,2025-05-30
40,AMPARO MONTOYA,12/11/2025,12/11/2025,NaN,29/09/2025,68540,No,"C/ de San Fermín avda 11, 2º Dcha. - 28041 MAD...",-818.68,NaN,DESCUENTO DE OBRA,Teleoperadora,Telemarketing,telemarketing,Amparo,Descuentos,1,2025-11-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8902,Vasilia,12/12/2025,12/12/2025,15/12/2025,21/10/2025,68616,No,"C/ San Maximiliano 23, 3ºC - 28017 MADRID (MA...",-71.00,NaN,DESCUENTO DE OBRA,Tecnico,Web,webs,Vasilia,Descuentos,1,2025-12-12
8907,Vasilia,18/12/2025,18/12/2025,17/07/2025,04/04/2025,67428,No,"C/ Gran Via del Sureste Avda 18, Esc A 3ºB - 2...",-517.08,18/12 X.O DE SANDRA SE METE DCTO** CLTE SE NIE...,DESCUENTO DE OBRA,Tecnico,Reformashoy,externos,Vasilia,Descuentos,1,2025-12-18
8908,Vasilia,18/12/2025,18/12/2025,14/10/2025,22/05/2025,67709,No,"C/ Gran Vía De Hortaleza 5E, 8ºB - 28033 MADR...",-1078.64,NaN,DESCUENTO DE OBRA,Tecnico,Web,webs,Vasilia,Descuentos,1,2025-12-18
8909,Vasilia,18/12/2025,18/12/2025,19/05/2025,04/04/2025,67455,No,"C/ Juan Bautista Monegro 10, portal 4, 4º B -...",-1249.65,X.O DE GLORIA METO DCTO POR MALA EJECUACION Y ...,DESCUENTO DE OBRA,Tecnico,Telemarketing,telemarketing,Vasilia,Descuentos,1,2025-12-18


In [89]:
ventas_totales25_check.to_excel("check.xlsx",index=False)

In [90]:
filtro = ventas_totales25[ventas_totales25["FAcepta fto fecha"] < "2026-01-01"]

In [91]:
filtro

,Comercial/Telemarketing/Técnico,Fecha Acepta,F.P.R.,Fecha Inicio,Fecha Entrega,PTO,Presupuesto Insitu,Dirección,Importe,Observaciones Pagos,Tipo,Departamento,Fuente,Origen,Empleado,Servicio,Column1,FAcepta fto fecha
5,AMPARO MONTOYA,03/03/2025,03/04/2025,28/03/2025,03/07/2024,65888,Sí,"C/ Puentelarra 8, Esc Izda 1º D - 28031 MADRID...",2306.00,"TOTAL 2.306 € + 21% IVA F.PAGO 10%,50%,40%",REFORMA BAÑO + MAMPARA,Teleoperadora,Telemarketing,telemarketing,Amparo,Reforma,1,2025-03-03
6,AMPARO MONTOYA,10/01/2025,10/02/2025,07/05/2025,10/01/2025,66834,No,"C/ Zazuar 7, 5ºD - 28031 MADRID (MADRID)",3005.00,TOTAL 3.005 € + 10% IVA (1º pto 2.953 €) F.PAG...,CARPINTERIA DE MADERA,Teleoperadora,Telemarketing,telemarketing,Amparo,Otros trabajos,1,2025-01-10
7,AMPARO MONTOYA,23/01/2025,15/02/2025,31/03/2025,16/01/2025,66948,No,"C/ Camino de los Alamillos 14, 1º A - 28891 V...",21547.58,"TOTAL 21.547,58 + 10%IVA F.PAGO 10%, 30%, 40% ...",REFORMA DE VIVIENDA 3%,Teleoperadora,Telemarketing,telemarketing,Amparo,Reforma,1,2025-01-23
8,AMPARO MONTOYA,18/02/2025,15/03/2025,10/04/2025,13/02/2025,66999,No,"C/ Europa Avda 17, Portal 8, 2º 2 - 28224 POZU...",16817.86,"TOTAL 16.817,86 €**CONTRATO PPAL DCTO 3% (50% ...",REFORMA DE VIVIENDA 3%,Teleoperadora,Telemarketing,telemarketing,Amparo,Reforma,1,2025-02-18
9,AMPARO MONTOYA,12/03/2025,12/04/2025,14/05/2025,26/02/2025,67128,No,"C/ Dr Fourquet 32, 2º izda - 28012 MADRID (MAD...",8372.00,TOTAL OBRA (8372 €) + LICENCIA (486 €) = 8.85...,REFORMA PINTURA Y VARIOS,Teleoperadora,Telemarketing,telemarketing,Amparo,Reforma,1,2025-03-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8915,Vasilia,18/12/2025,18/01/2026,NaN,17/11/2025,68827,No,"C/ Fernando Delgado 8, 6º B - 28047 MADRID (M...",4011.85,NaN,ENCIMERA,Tecnico,Web,webs,Vasilia,Otros trabajos,1,2025-12-18
8916,Vasilia,18/12/2025,18/01/2026,NaN,17/11/2025,68827,No,"C/ Fernando Delgado 8, 6º B - 28047 MADRID (M...",569.05,NaN,REFORMA PVC SLAVA,Tecnico,Web,webs,Vasilia,Reforma,1,2025-12-18
8917,Vasilia,22/12/2025,22/01/2026,NaN,12/12/2025,68930,No,"C/ Nerja 3, esc 1, 2º A - 28045 MADRID (MADRID)",165.00,NaN,CONTENEDOR,Tecnico,Web,webs,Vasilia,Contenedores,1,2025-12-22
8918,Vasilia,06/11/2025,02/02/2026,NaN,09/10/2025,68584,No,"C/ Corregidor Juan Fco de Lujan 99, 1ºA - 280...",261.00,REGULARIZAR EN INICIO DE OBRA,AMPLIACION DE OBRA,Tecnico,Rdp,webs,Vasilia,Ampliaciones,1,2025-11-06


In [92]:
filtro.drop("FAcepta fto fecha",axis=1,inplace=True)

In [93]:
filtro

,Comercial/Telemarketing/Técnico,Fecha Acepta,F.P.R.,Fecha Inicio,Fecha Entrega,PTO,Presupuesto Insitu,Dirección,Importe,Observaciones Pagos,Tipo,Departamento,Fuente,Origen,Empleado,Servicio,Column1
5,AMPARO MONTOYA,03/03/2025,03/04/2025,28/03/2025,03/07/2024,65888,Sí,"C/ Puentelarra 8, Esc Izda 1º D - 28031 MADRID...",2306.00,"TOTAL 2.306 € + 21% IVA F.PAGO 10%,50%,40%",REFORMA BAÑO + MAMPARA,Teleoperadora,Telemarketing,telemarketing,Amparo,Reforma,1
6,AMPARO MONTOYA,10/01/2025,10/02/2025,07/05/2025,10/01/2025,66834,No,"C/ Zazuar 7, 5ºD - 28031 MADRID (MADRID)",3005.00,TOTAL 3.005 € + 10% IVA (1º pto 2.953 €) F.PAG...,CARPINTERIA DE MADERA,Teleoperadora,Telemarketing,telemarketing,Amparo,Otros trabajos,1
7,AMPARO MONTOYA,23/01/2025,15/02/2025,31/03/2025,16/01/2025,66948,No,"C/ Camino de los Alamillos 14, 1º A - 28891 V...",21547.58,"TOTAL 21.547,58 + 10%IVA F.PAGO 10%, 30%, 40% ...",REFORMA DE VIVIENDA 3%,Teleoperadora,Telemarketing,telemarketing,Amparo,Reforma,1
8,AMPARO MONTOYA,18/02/2025,15/03/2025,10/04/2025,13/02/2025,66999,No,"C/ Europa Avda 17, Portal 8, 2º 2 - 28224 POZU...",16817.86,"TOTAL 16.817,86 €**CONTRATO PPAL DCTO 3% (50% ...",REFORMA DE VIVIENDA 3%,Teleoperadora,Telemarketing,telemarketing,Amparo,Reforma,1
9,AMPARO MONTOYA,12/03/2025,12/04/2025,14/05/2025,26/02/2025,67128,No,"C/ Dr Fourquet 32, 2º izda - 28012 MADRID (MAD...",8372.00,TOTAL OBRA (8372 €) + LICENCIA (486 €) = 8.85...,REFORMA PINTURA Y VARIOS,Teleoperadora,Telemarketing,telemarketing,Amparo,Reforma,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8915,Vasilia,18/12/2025,18/01/2026,NaN,17/11/2025,68827,No,"C/ Fernando Delgado 8, 6º B - 28047 MADRID (M...",4011.85,NaN,ENCIMERA,Tecnico,Web,webs,Vasilia,Otros trabajos,1
8916,Vasilia,18/12/2025,18/01/2026,NaN,17/11/2025,68827,No,"C/ Fernando Delgado 8, 6º B - 28047 MADRID (M...",569.05,NaN,REFORMA PVC SLAVA,Tecnico,Web,webs,Vasilia,Reforma,1
8917,Vasilia,22/12/2025,22/01/2026,NaN,12/12/2025,68930,No,"C/ Nerja 3, esc 1, 2º A - 28045 MADRID (MADRID)",165.00,NaN,CONTENEDOR,Tecnico,Web,webs,Vasilia,Contenedores,1
8918,Vasilia,06/11/2025,02/02/2026,NaN,09/10/2025,68584,No,"C/ Corregidor Juan Fco de Lujan 99, 1ºA - 280...",261.00,REGULARIZAR EN INICIO DE OBRA,AMPLIACION DE OBRA,Tecnico,Rdp,webs,Vasilia,Ampliaciones,1


In [94]:
#excluyo los 3000000
filtro = filtro[~filtro["PTO"].astype(str).str.startswith("300")]


In [95]:
filtro.to_excel("ventas2025.xlsx",index=False)